<a href="https://colab.research.google.com/github/Jiablero/notebooks/blob/master/titanic_kaggle_contest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# gender = pd.read_csv('gender_submission.csv') # just an example of result file

Data Preparation.




In [0]:
def prepare_titanic_data(data):
  X = data.drop(['PassengerId', 'Name', 'Fare', 'Ticket', 'Embarked', 'Cabin'], axis = 1)
  if 'Survived' in data.columns : 
    X = X.drop('Survived', axis = 1) # в тестовых данных нет Survived
  X = X.fillna({'Age': X.Age.median()})
  X = pd.get_dummies(X).drop('Sex_male', axis=1)
  return(X)

In [0]:
X, test_X = train_test_split(train, test_size = 0.2)

In [0]:
y = X.Survived
X = prepare_titanic_data(X)
test_y = test_X.Survived
test_X = prepare_titanic_data(test_X)

Classifiers creation, training.

In [0]:
def grid_search(clf, params, X, y):
  gs = GridSearchCV(clf, params, n_jobs=-1)
  gs.fit(X, y)
  # best_params_ для отладки
  return {'best_params_': gs.best_params_, 'best_estimator_': gs.best_estimator_}

In [0]:
# Random Forest
rf_clf = RandomForestClassifier()
rf_parameters = {
    'n_estimators': range(10, 30, 5), 
    'max_depth': range(3, 10, 1), 
    'min_samples_leaf': range(1, 7), 
    'min_samples_split': range(4, 12, 2)}

In [0]:
clf_final_rf = grid_search(rf_clf, rf_parameters, X, y)

In [291]:
clf_final_rf.get('best_estimator_').score(X, y)

0.824438202247191

In [0]:
# Neural network (MLP)
nn_clf = MLPClassifier()
nn_parameters = {  'activation': ['identity', 'logistic', 'tanh', 'relu'],
                'solver': ['lbfgs', 'sgd', 'adam'],
                'alpha': np.arange(0.0001, 0.0005, 0.0001),
                'learning_rate': ['constant', 'invscaling', 'adaptive'],
                'max_iter': range(100, 500, 100),
                'validation_fraction': np.arange(0.1, 0.4, 0.1), 
              }

In [0]:
# clf_final_nn = grid_search(nn_clf, nn_parameters, X, y)

In [0]:
# clf_final_nn.get('best_params_')
# Чтобы не считать каждый раз по часу, сохранил best_params_

clf_final_by['best_params_'] = {'activation': 'tanh',
                                'alpha': 0.0002,
                                'learning_rate': 'invscaling',
                                'max_iter': 300,
                                'solver': 'adam',
                                'validation_fraction': 0.1}

In [0]:
nn_fixed_clf = MLPClassifier( activation = 'tanh',alpha = 0.0002, 
                              learning_rate = 'invscaling',
                              max_iter = 300,
                              solver = 'adam',
                              validation_fraction = 0.1)

In [296]:
nn_fixed_clf.fit(X, y)

MLPClassifier(activation='tanh', alpha=0.0002, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='invscaling',
              learning_rate_init=0.001, max_fun=15000, max_iter=300,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [297]:
#clf_final_nn.get('best_estimator_').score(X, y)
nn_fixed_clf.score(X, y)

0.827247191011236

In [0]:
# Bayes. Надо бы CategoricalNB, но он глючит.
by_clf = MultinomialNB()
by_parameters = {'alpha': np.arange(0.0, 2.0, 0.1)}

In [299]:
clf_final_by = grid_search(by_clf, by_parameters, X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [300]:
clf_final_by.get('best_params_')

{'alpha': 0.0}

In [301]:
clf_final_by.get('best_estimator_').score(X, y)

0.7865168539325843

**Results, Consensus**

Попытка решить задачу используя консенсус 3-х методов (random forest, bayes, nn).

Особого выигрыша по сравнению с одной nn это не принесло:

 +- 0.5% результата.

In [302]:
clf_final_rf.get('best_estimator_').score(test_X, test_y)

0.8268156424581006

In [303]:
nn_fixed_clf.score(test_X, test_y)

0.8324022346368715

In [304]:
clf_final_by.get('best_estimator_').score(test_X, test_y)

0.770949720670391

In [0]:
rf = pd.DataFrame(data = clf_final_rf.get('best_estimator_').predict_proba(test_X))
nn = pd.DataFrame(data = nn_fixed_clf.predict_proba(test_X))
by = pd.DataFrame(data = clf_final_by.get('best_estimator_').predict_proba(test_X))

In [0]:
test_result = pd.DataFrame(data={'rf_1': rf[1], 'nn_1': nn[1], 'by_1': by[1]})

In [307]:
test_result

,rf_1,nn_1,by_1
0,0.851916,0.822269,0.619381
1,0.524478,0.567657,0.549617
2,0.388220,0.305913,0.237093
3,0.829570,0.630080,0.741144
4,0.226410,0.135934,0.278542
...,...,...,...
174,0.136669,0.079937,0.205231
175,0.719339,0.980997,0.701069
176,0.792993,0.889049,0.590319
177,0.524478,0.561546,0.544499


In [0]:
test_result['consensus'] = (test_result.rf_1 + test_result.nn_1 + test_result.by_1) / 3
test_result.consensus = test_result.consensus.round()


In [0]:
test_result['real_result'] = test_y.values

In [0]:
test_result['match'] = np.where(test_result.consensus == test_result.real_result, 1, 0)

In [311]:
# +- 0.005 по сравнению с одной нейросетью :/
test_result.match.mean()

0.8268156424581006

In [312]:
test_result

,rf_1,nn_1,by_1,consensus,real_result,match
0,0.851916,0.822269,0.619381,1.0,1,1
1,0.524478,0.567657,0.549617,1.0,0,0
2,0.388220,0.305913,0.237093,0.0,0,1
3,0.829570,0.630080,0.741144,1.0,1,1
4,0.226410,0.135934,0.278542,0.0,0,1
...,...,...,...,...,...,...
174,0.136669,0.079937,0.205231,0.0,0,1
175,0.719339,0.980997,0.701069,1.0,0,0
176,0.792993,0.889049,0.590319,1.0,1,1
177,0.524478,0.561546,0.544499,1.0,1,1


In [0]:
#result = pd.DataFrame({'PassengerID': test.PassengerId, 'Survived': test_result.consensus})

In [0]:
#result

In [0]:
# TODO: добавить еще пару методов, отрефакторить, оттестить.